In [ ]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

es=Elasticsearch([{"host":"localhost","port":9200}],http_auth=("elastic","suVlWmYRKKFk6RYs_TrU"))
print(es.ping())

df=pd.read_csv('buildler2.csv')

df = df.where(pd.notnull(df), None)

path_of_exile_builds_index = {
    "settings": {
        "index": {
            "analysis": {
                "analyzer": {
                    "custom_analyzer": {
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": [
                            "lowercase",
                            "asciifolding",
                            "custom_edge_ngram"
                        ]
                    }
                },
                "filter": {
                    "custom_edge_ngram": {
                        "type": "edge_ngram",
                        "min_gram": 2,
                        "max_gram": 10
                    }
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "Build": {
                "type": "text",
                "analyzer": "custom_analyzer"
            },
            "Ascendancy": {
                "type": "keyword"
            },
            "Açıklama": {
                "type": "text",
                "analyzer": "custom_analyzer"
            }
        }
    }
}

def dataframe_to_es(df, es_index):
    for df_idx, line in df.iterrows():
        yield {
            "_index": es_index,
            "_id": df_idx,  # Satır numarasını Elasticsearch belgesi ID'si olarak kullanıyoruz
            "_source": {
                "Build": line['Build'],
                "Ascendancy": line['Ascendancy'],
                "Açıklama": line['Açıklama']
            }
        }

try:
    es.indices.delete("path_of_exile_builds_index")
except:
    print("No index")

es.indices.create(index="path_of_exile_builds_index", body=path_of_exile_builds_index)  # <--- Eklenen satır

bulk(es, dataframe_to_es(df, "path_of_exile_builds_index"), raise_on_error=False)

search_query = " new jugg " #ENTER THE SEARCH INPUT HERE

response = es.search(index='path_of_exile_builds_index', body={
    "query": {
        "match": {
            "Build": {
                "query": search_query,
                "operator": "and"  # <--- Eklenen satır
            }
        }
    }
})

for hit in response['hits']['hits']:
    print(hit['_source'])

: 